In [152]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [1]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.signal as sig

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$

In [2]:
# Parameter
m  = 1
w  = 2
Q = la.block_diag(3,4,5)
R = 6

In [48]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([0,1/m,0])
B_ = np.array([[0,1/m,0]])

C = np.array([1,0,0])
D = 0


#Ad,Bd,Cd,Dd,Ta = sig.cont2discrete((A,B,C,D),0.1, method='foh')
Ad,Bd,Cd,Dd,Ta = sig.cont2discrete((A,B_.T,C,D),0.01, method='foh')

In [45]:
N = 10000
nx = 3


def rollout(k,x):
    u     = k@x    
    dx     = Ad@x + Bd.T*u    
    #dx    =A@x +B*u
    return dx , u

def cost(k):    
    x0 = np.array([1,0,0])   
    
    x_new = np.zeros((nx,N))
    u = np.zeros((1,N))
    
    x_new[:,1], u[:,0] = rollout(k,x0)  

    cost_ = x0@Q@x0*0 + R*u[:,0]**2*0
   
    for i in range(1,N-2): 
        # laufende Kosten
        x_new[:,i+1], u[:,i] = rollout(k,x_new[:,i])       
        
        cost_ += x_new[:,i+1]@Q@x_new[:,i+1]
        cost_ += u[:,i]**2*R
   
    return cost_   

print(cost([-1,-1,-1]))   



[2696.54608729]


In [47]:

P = la.solve_continuous_are(A,B_.T,Q,R)
K = -B@P/R
print(K)
Pd = la.solve_discrete_are(Ad,Bd,Q,R)
Kd = -Bd.T@Pd/R
print(Kd)

[-0.30926715 -1.13366704 -0.91287093]
[[-0.3473642  -1.13682087 -0.91805992]]


In [46]:

from scipy.optimize import minimize
k0 = np.array([0, -1, -1])
res = minimize(cost, k0, method="nelder-mead", options={"disp": True})
print(res)
print(cost(res.x))

Optimization terminated successfully.
         Current function value: 2386.143945
         Iterations: 117
         Function evaluations: 200
 final_simplex: (array([[-0.31450755, -1.13093876, -0.91570986],
       [-0.31452033, -1.13097526, -0.91578623],
       [-0.31448333, -1.13095449, -0.91574541],
       [-0.31450629, -1.13095777, -0.91568168]]), array([2386.14394456, 2386.14394476, 2386.14394492, 2386.14394501]))
           fun: 2386.1439445619235
       message: 'Optimization terminated successfully.'
          nfev: 200
           nit: 117
        status: 0
       success: True
             x: array([-0.31450755, -1.13093876, -0.91570986])
[2386.14394456]
